In [16]:
import networkx as nx

import KnowledgeGraph as kg  # type: ignore

from multiprocessing import Pool
import mp_for_nb


import pickle

In [2]:
drug_terms=set([
    "lacosamide",
    "fluoxetine",
    "calcium",
    "zolpidem",
    "cannabis",
    "hydroxyzine",
    "pregabalin",
    "sumatriptan",
    "estradiol",
    "vagal nerve stimulator implantation",
    "citalopram",
    "prazosin",
    "gabapentin",
    "olanzapine",
    "ethosuximide",
    "dextroamphetamine",
    "valproic acid",
    "escitalopram",
    "levetiracetam",
    "bisoprolol",
    "alprazolam",
    "propranolol",
    "tiagabine",
    "fluvoxamine",
    "trazodone",
    "duloxetine",
    "folic acid",
    "amitriptyline",
    "cenobamate",
    "cbd",
    "phenobarbital",
    "diazepam",
    "verapamil",
    "levonorgestrel",
    "chlordiazepoxide",
    "sertraline",
    "vigabatrin",
    "venlafaxine",
    "eslicarbazepine acetate",
    "lisdexamfetamine",
    "acetazolamide",
    "lamotrigine",
    "primidone",
    "aripiprazole",
    "oxcarbazepine",
    "clobazam",
    "carbamazepine",
    "quetiapine",
    "lorazepam",
    "diltiazem",
    "perampanel",
    "sulthiame",
    "buspirone",
    "nortriptyline",
    "brivaracetam",
    "caffeine",
    "zonisamide",
    "phenytoin",
    "bupropion",
    "topiramate",
    "clonazepam",
])

In [3]:
G_EFF = nx.read_graphml('Sample Epilepsy Graphs/04-efwebsite-forums-network-20180706-samepost_all_users.graphml')
G_TWI = nx.read_graphml('Sample Epilepsy Graphs/04-twitter-epilepsy-network-20180706-samepost-all-users.graphml')
G_INS = nx.read_graphml('Sample Epilepsy Graphs/04-instagram-epilepsy-network-20180706-samepost-all-users.graphml')
G_RED = nx.read_graphml('Sample Epilepsy Graphs/reddit_network.graphml')

Layers = {
    'EFF' : G_EFF,
    'Twitter' : G_TWI,
    'Instagram' : G_INS,
    'Reddit' : G_RED,
    }

all_nodes = {x:d for G in Layers.values() for x,d in G.nodes(data=True)}

for node,data in all_nodes.items():
    for G in Layers.values():
        if node not in G.nodes():
            G.add_nodes_from([(node,data)])
        if data['parent'].lower() not in drug_terms:
            G.remove_node(node)
            
all_edges = set()
for L,G in Layers.items():
    all_edges |= set(((u,L),(v,L)) for u,v in G.edges()) 
    all_edges |= set(((v,L),(u,L)) for u,v in G.edges())
total_edges = len(all_edges)//2
print(f'{total_edges=}')

total_edges=1503


In [4]:
KG = kg.KnowledgeGraph(Layers)

In [7]:
def appd(arg):
    G = arg[0]
    n = arg[1]
    c = arg[2]
    return n.id, G.pareto_shortest_distances(n,depth_cut=c,weight='proximity')

In [8]:
arglist = mp_for_nb.make_arglist(KG,'EFF',cut=7)

with Pool() as pool:
    results = pool.imap_unordered(appd,arglist)
    APPD = list(results)
        
closure = {k[0]: {vk[0]: vv for vk, vv in v.items()} for k, v in APPD}


In [17]:
with open("Epilepsy_MultiClosure.pickle", "wb") as f:
        pickle.dump(closure, f, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
len(closure)

57

In [12]:
removed_edges_mp = KG.pareto_backbone_removed_edges(closure=closure,weight='proximity')

In [15]:
sre = set(removed_edges_mp | set((v,u) for u,v in removed_edges_mp))
print(len(sre)//2)

1041
